In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

from pathlib import Path

In [2]:
COLOR_CODE = {
    "_background_": [0, 0, 0],
    "Building": [128, 0, 0],
    "Pedestrian": [0, 128, 0],
    "Car": [0, 0, 128],
    "Sidewalk": [128, 0, 128],
    "Vegetation": [0, 128, 128],
    "ManmadeStructure": [64, 0, 0],
    "Road": [192, 0, 0],
    "Sky": [64, 128, 0],
    "SignSymbol": [192, 128, 0],
    "SignPole": [64, 0, 128],
    "Billboard": [128, 0, 128],
}

In [3]:
IMAGE_PATH = 'I:\Sample100\data_dataset_voc\JPEGImages'
MASK_PATH = 'I:\Sample100\data_dataset_voc\SegmentationClassPNG'

IMAGE_SIZE = (256, 256)
N_CLASSES = len(COLOR_CODE)

BATCH_INPUT_DIM = (None, 256, 256, 3) 
BATCH_OUTPUT_DIM = (None, 256, 256, N_CLASSES)
BATCH_SIZE = 3

In [ ]:
def read_images_and_masks(image_path, mask_path):
    images = []
    masks = []
    file_list = np.sort(os.listdir(IMAGE_PATH))
    for filename in file_list:
        image_path = os.path.join(IMAGE_PATH, filename)
        mask_path = os.path.join(MASK_PATH, filename[:-4] + '.png')
        try:
            image = cv2.imread(image_path)
            mask = cv2.imread(mask_path)
            
            image = cv2.resize(image, IMAGE_SIZE)
            mask = cv2.resize(mask, IMAGE_SIZE)
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
            
            image = image / 255.0
            
            images.append(image)
            masks.append(mask)
        except:
            continue
    
    return images, masks

images, masks = read_images_and_masks(IMAGE_PATH, MASK_PATH)

In [4]:
def get_masks_one_hot(mask, color_codes):
    n_channels = len(color_codes.keys())  # N_CLASSES
    mask_channels = np.zeros((mask.shape[0], mask.shape[1], n_channels), 
                             dtype=np.float32)
    for i, cls in enumerate(color_codes.keys()):
        color = color_codes[cls]
        sub_mask = np.all(mask==color, axis=-1) * 1
        mask_channels[:, :, i] = sub_mask
    return mask_channels

def tf_data_generator(file_list, image_path, mask_path, batch_size = 10):
    i = 0
    while True:
        if i * batch_size > len(file_list):
            i = 0
            np.random.shuffle(file_list)
        else:
            images = []
            masks = []
            file_chunk = file_list[i * batch_size:(i + 1) * batch_size]
            i = i + 1
            for filename in file_chunk:
                path_to_image = os.path.join(image_path, filename)
                path_to_mask = os.path.join(mask_path, filename[:-4])
                
                if isinstance(path_to_image, (bytearray, bytes)):
                    path_to_image = path_to_image.decode('utf-8')
                    path_to_mask = path_to_mask.decode('utf-8') + '.png'
                
                elif isinstance(path_to_image, str):
                    path_to_image = str(path_to_image)
                    path_to_mask = str(path_to_mask) + '.png'
                    
                try:
                    image = cv2.imread(path_to_image)
                    mask = cv2.imread(path_to_mask)
                    
                    image = cv2.resize(image, IMAGE_SIZE)
                    mask = cv2.resize(mask, IMAGE_SIZE)
                    
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
                    
                    images.append(image)
                    masks.append(mask)
                except:
                    continue
            
            masks_one_hot = [get_masks_one_hot(mask, COLOR_CODE) for mask in masks]
            
            images = np.asarray(images).reshape(-1, IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
            masks_one_hot = np.asarray(masks_one_hot).reshape(-1, IMAGE_SIZE[0], IMAGE_SIZE[1], N_CLASSES)
            yield images, masks_one_hot

In [5]:
file_list = np.sort(os.listdir(IMAGE_PATH))
batch_generator = tf_data_generator(file_list, image_path=IMAGE_PATH, mask_path=MASK_PATH, batch_size=BATCH_SIZE)
num = 0
for images, masks in batch_generator:
    # plt.figure(figsize=(20,20))
    # plt.subplot(1,2,1)
    # plt.imshow(images[0])
    # plt.axis('off')

    # plt.subplot(1,2,2)
    # plt.imshow(masks[0][:,:,9])
    # plt.axis('off')
    # plt.show()
    
    print(f'{images.shape} {masks.shape}')
    num = num + 1
    if num > 5: break

(3, 256, 256, 3) (3, 256, 256, 12)
(3, 256, 256, 3) (3, 256, 256, 12)
(3, 256, 256, 3) (3, 256, 256, 12)
(3, 256, 256, 3) (3, 256, 256, 12)
(3, 256, 256, 3) (3, 256, 256, 12)
(3, 256, 256, 3) (3, 256, 256, 12)


In [6]:
file_list = np.sort(os.listdir(IMAGE_PATH))
dataset = tf.data.Dataset.from_generator(tf_data_generator,
                                         args=[file_list, IMAGE_PATH, MASK_PATH, 6],
                                         output_types=(tf.float32, tf.float32),
                                         output_shapes=(BATCH_INPUT_DIM, BATCH_OUTPUT_DIM))
for images, masks in dataset.take(5):
    print(f'{images.shape} {masks.shape}')

(6, 256, 256, 3) (6, 256, 256, 12)
(6, 256, 256, 3) (6, 256, 256, 12)
(6, 256, 256, 3) (6, 256, 256, 12)
(6, 256, 256, 3) (6, 256, 256, 12)
(6, 256, 256, 3) (6, 256, 256, 12)
